We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [1]:
import os
from pathlib import Path
import duckdb

print("cwd", Path().resolve())

candidates = []
search_roots = [
    Path("."),
    Path(".."),
    Path("./data"),
    Path("../data"),
    Path("../../data"),
]
for root in search_roots:
    if root.exists():
        candidates.extend(list(root.glob("*.duckdb")))
        candidates.extend(list(root.glob("**/*.duckdb")))

seen = set()
duckdb_files = []
for f in candidates:
    fp = str(f.resolve())
    if fp not in seen:
        seen.add(fp)
        duckdb_files.append(f.resolve())

print("duckdb files found")
for i, f in enumerate(duckdb_files[:25]):
    print(i, f)

db_file = None
for f in duckdb_files:
    if f.name == "nflpa.duckdb":
        db_file = f
        break

if db_file is None and duckdb_files:
    db_file = duckdb_files[0]

if db_file is None:
    raise RuntimeError("No duckdb file found near this notebook, rerun notebook 02 or check where you stored the database file")

con = duckdb.connect(str(db_file))
print("connected db", db_file)

tables = con.execute("SHOW TABLES").df()
tables

cwd /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/notebooks
duckdb files found
0 /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/db/nflpa.duckdb
1 /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/data/nflpa.duckdb
connected db /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/db/nflpa.duckdb


,name
0,injuries
1,injuries_players_distinct
2,injuries_team_week_players
3,injury_counts_next
4,injury_counts_next_game
5,injury_counts_next_game_with_lags
6,injury_counts_with_lags
7,injury_outcomes_next_game_tmp
8,injury_outcomes_tmp
9,injury_players_next_game


We define small helpers for column discovery and strict required column checks so later sanity checks fail early if an upstream notebook was skipped or a column name drifted

In [2]:
import pandas as pd

SEASON_COL = "season"
WEEK_COL = "week"
TEAM_COL = "team"
PANEL_TABLE = "team_week_panel"

def _existing_cols(table_name: str) -> list[str]:
    return con.execute(f"DESCRIBE {table_name}").df()["column_name"].tolist()

def _require_cols(table_name: str, required: list[str]) -> None:
    cols = set(_existing_cols(table_name))
    missing = [c for c in required if c not in cols]
    print("Missing required columns", missing)
    if missing:
        raise RuntimeError(f"Missing columns in {table_name}, rerun earlier notebooks, missing, {missing}")

def _cols_matching(prefix: str = "", contains: str = "", suffix: str = "") -> list[str]:
    cols = _existing_cols(PANEL_TABLE)
    out = []
    for c in cols:
        if prefix and not c.startswith(prefix):
            continue
        if contains and contains not in c:
            continue
        if suffix and not c.endswith(suffix):
            continue
        out.append(c)
    return out

print("team week panel columns", len(_existing_cols(PANEL_TABLE)))

team week panel columns 152


Quick sanity check to confirm that the wide-format structure correctly reflects the expected mix of injury, workload, and control features

In [3]:
con.execute("DESCRIBE team_week_panel").df().head(30)

con.execute("""
SELECT
  column_name,
  column_type
FROM (DESCRIBE team_week_panel)
ORDER BY column_name
LIMIT 30
""").df()

,column_name,column_type
0,Cum_Shocks_All_w,HUGEINT
1,Cum_Shocks_NonScore_w,HUGEINT
2,Cum_Shocks_ScoreLinked_w,HUGEINT
3,Cumulative_Workload_Index_pg_w,DOUBLE
4,Cumulative_Workload_Index_w,DOUBLE
5,Inj_Def_LastGame_w,DOUBLE
6,Inj_Def_Last_w,DOUBLE
7,Inj_Def_NextGame_w,DOUBLE
8,Inj_Def_Next_w,DOUBLE
9,Inj_Off_LastGame_w,DOUBLE


Quick sanity check to confirm that both schema views contain the same columns and types

In [4]:
df_a = con.execute("DESCRIBE team_week_panel").df()
df_b = con.execute("""
SELECT column_name, column_type
FROM (DESCRIBE team_week_panel)
ORDER BY column_name
""").df()

print("rows describe", len(df_a))
print("rows sorted", len(df_b))

dupe_names = df_a["column_name"].value_counts()
dupe_names = dupe_names[dupe_names > 1]
print("duplicate column names", dupe_names.to_dict())

mismatch = (
    df_a[["column_name", "column_type"]]
    .drop_duplicates()
    .merge(
        df_b[["column_name", "column_type"]].drop_duplicates(),
        on=["column_name", "column_type"],
        how="outer",
        indicator=True,
    )
)
mismatch = mismatch[mismatch["_merge"] != "both"]
mismatch.head(50)

rows describe 152
rows sorted 152
duplicate column names {}


,column_name,column_type,_merge


We verify the continuity of the dataset by aggregating the unique time periods present to ensure that the panel spans the full historical range required for the analysis

In [5]:
con.execute(f"""
SELECT
  SUM(CASE WHEN name = '{PANEL_TABLE}' THEN 1 ELSE 0 END) AS has_team_week_panel
FROM (SHOW TABLES)
""").df()

_require_cols(
    PANEL_TABLE,
    [
        SEASON_COL,
        WEEK_COL,
        TEAM_COL,
        "game_id",
        "points_for",
        "points_against",
        "ST_Load_All_w",
        "ST_Load_ScoreLinked_w",
        "ST_Load_NonScore_w",
        "ST_Punt_w",
        "ST_PuntReturn_w",
        "ST_Kickoff_w",
        "ST_KickReturn_w",
        "ST_FG_w",
        "ST_XP_w",
        "ST_Rare_w",
        "Inj_Off_Next_w",
        "Inj_Def_Next_w",
    ],
)

con.execute(f"""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT {SEASON_COL}) AS seasons,
  MIN({SEASON_COL}) AS min_season,
  MAX({SEASON_COL}) AS max_season,
  MIN({WEEK_COL}) AS min_week,
  MAX({WEEK_COL}) AS max_week
FROM {PANEL_TABLE}
""").df()

Missing required columns []


,rows,seasons,min_season,max_season,min_week,max_week
0,6782,13,2012,2024,1,18


Quick sanity check to confirm that 'team_week_panel' exists in the connected database 

In [6]:
con.execute("""
SELECT
  SUM(CASE WHEN name = 'team_week_panel' THEN 1 ELSE 0 END) AS has_team_week_panel
FROM (SHOW TABLES)
""").df()

,has_team_week_panel
0,1.0


Quick sanity check to confirm that 'team_week_panel' has no duplicate season-week-team rows and that the key count equals the row count to ensure that the final modeling table is a perfectly unique panel

In [7]:
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT season || '-' || week || '-' || team) AS distinct_keys
FROM team_week_panel
""").df()

dups = con.execute("""
SELECT
  season,
  week,
  team,
  COUNT(*) AS n
FROM team_week_panel
GROUP BY 1,2,3
HAVING COUNT(*) > 1
ORDER BY n DESC, season, week, team
LIMIT 50
""").df()

dups

if len(dups) > 0:
    raise RuntimeError("Duplicate season week team rows exist in team_week_panel, investigate joins before proceeding")

We verify that the "next game" indicator accurately distinguishes between teams that play in the following calendar week and those heading into a bye to ensure that the 'w+a' outcome logic aligns with the physical game schedule

In [8]:
con.execute("DROP VIEW IF EXISTS panel_next_week_flags")

con.execute(f"""
CREATE TEMP VIEW panel_next_week_flags AS
WITH base AS (
  SELECT
    {SEASON_COL} AS season,
    {WEEK_COL} AS week,
    CAST({TEAM_COL} AS VARCHAR) AS team_key
  FROM {PANEL_TABLE}
),
nxt AS (
  SELECT
    {SEASON_COL} AS season,
    {WEEK_COL} AS week,
    CAST({TEAM_COL} AS VARCHAR) AS team_key
  FROM {PANEL_TABLE}
)
SELECT
  b.season,
  b.week,
  b.team_key,
  CASE WHEN n.week IS NULL THEN 0 ELSE 1 END AS has_next_week
FROM base b
LEFT JOIN nxt n
  ON n.season = b.season
 AND n.team_key = b.team_key
 AND n.week = b.week + 1
""")

con.execute("""
SELECT
  has_next_week,
  COUNT(*) AS n_rows
FROM panel_next_week_flags
GROUP BY 1
ORDER BY 1
""").df()

,has_next_week,n_rows
0,0,832
1,1,5950


Quick sanity check to confirm that next week injury outcomes are only defined when week 'w+a' exists for that team season and that they are null when the next week row is missing

In [9]:
inj_next_week_cols = ["Inj_Off_Next_w", "Inj_Def_Next_w"]
cols_now = set(_existing_cols(PANEL_TABLE))
inj_next_week_cols = [c for c in inj_next_week_cols if c in cols_now]

if not inj_next_week_cols:
    raise RuntimeError("No next week injury outcome columns found, expected Inj_Off_Next_w and Inj_Def_Next_w")

print("Next week injury columns checked", inj_next_week_cols)

select_any_defined = " OR ".join([f"p.{c} IS NOT NULL" for c in inj_next_week_cols])

bad_defined = con.execute(f"""
SELECT
  COUNT(*) AS bad_rows
FROM {PANEL_TABLE} p
JOIN panel_next_week_flags f
  ON f.season = p.{SEASON_COL}
 AND f.week = p.{WEEK_COL}
 AND f.team_key = CAST(p.{TEAM_COL} AS VARCHAR)
WHERE f.has_next_week = 0
  AND ({select_any_defined})
""").df()

bad_rows = int(bad_defined["bad_rows"].iloc[0])
print(bad_defined)

if bad_rows != 0:
    sample = con.execute(f"""
    SELECT
      p.{SEASON_COL} AS season,
      p.{WEEK_COL} AS week,
      p.{TEAM_COL} AS team,
      f.has_next_week,
      {", ".join([f"p.{c}" for c in inj_next_week_cols])}
    FROM {PANEL_TABLE} p
    JOIN panel_next_week_flags f
      ON f.season = p.{SEASON_COL}
     AND f.week = p.{WEEK_COL}
     AND f.team_key = CAST(p.{TEAM_COL} AS VARCHAR)
    WHERE f.has_next_week = 0
      AND ({select_any_defined})
    ORDER BY p.{SEASON_COL} DESC, p.{WEEK_COL} DESC, p.{TEAM_COL}
    LIMIT 50
    """).df()
    print(sample)
    raise RuntimeError("Next week injury outcomes are populated when week plus 1 does not exist, fix notebook 09 logic or apply the step 11 repair cell")

Next week injury columns checked ['Inj_Off_Next_w', 'Inj_Def_Next_w']
   bad_rows
0         0


We handle the undefined next week cases by creating a modeling view that drops all rows where week 'w+1' does not exist, which are the final game weeks and the pre bye game weeks.

In [10]:
con.execute("DROP VIEW IF EXISTS team_week_panel_nextweek_model")

con.execute(f"""
CREATE VIEW team_week_panel_nextweek_model AS
SELECT
  p.*,
  f.has_next_week
FROM {PANEL_TABLE} p
JOIN panel_next_week_flags f
  ON f.season = p.{SEASON_COL}
 AND f.week = p.{WEEK_COL}
 AND f.team_key = CAST(p.{TEAM_COL} AS VARCHAR)
WHERE f.has_next_week = 1
""")

con.execute(f"""
SELECT
  COUNT(*) AS rows_model,
  COUNT(DISTINCT {SEASON_COL} || '-' || {WEEK_COL} || '-' || {TEAM_COL}) AS distinct_keys_model
FROM team_week_panel_nextweek_model
""").df()

con.execute(f"""
SELECT
  p.{SEASON_COL} AS season,
  COUNT(*) AS rows_total,
  SUM(CASE WHEN f.has_next_week = 1 THEN 1 ELSE 0 END) AS rows_kept,
  SUM(CASE WHEN f.has_next_week = 0 THEN 1 ELSE 0 END) AS rows_dropped
FROM {PANEL_TABLE} p
JOIN panel_next_week_flags f
  ON f.season = p.{SEASON_COL}
 AND f.week = p.{WEEK_COL}
 AND f.team_key = CAST(p.{TEAM_COL} AS VARCHAR)
GROUP BY 1
ORDER BY 1
""").df()

,season,rows_total,rows_kept,rows_dropped
0,2012,512,448.0,64.0
1,2013,512,448.0,64.0
2,2014,512,448.0,64.0
3,2015,512,448.0,64.0
4,2016,512,448.0,64.0
5,2017,512,450.0,62.0
6,2018,512,448.0,64.0
7,2019,512,448.0,64.0
8,2020,512,448.0,64.0
9,2021,544,480.0,64.0


Quick sanity check to confirm that the special teams totals exactly match the intended bucket identities

In [11]:
st_identity = con.execute(f"""
SELECT
  SUM(CASE WHEN ST_Load_All_w != (ST_Punt_w + ST_PuntReturn_w + ST_Kickoff_w + ST_KickReturn_w + ST_FG_w + ST_XP_w + ST_Rare_w) THEN 1 ELSE 0 END) AS bad_all_identity,
  SUM(CASE WHEN ST_Load_ScoreLinked_w != (ST_Kickoff_w + ST_XP_w + ST_FG_w) THEN 1 ELSE 0 END) AS bad_scorelinked_identity,
  SUM(CASE WHEN ST_Load_NonScore_w != (ST_Punt_w + ST_PuntReturn_w + ST_KickReturn_w + ST_Rare_w) THEN 1 ELSE 0 END) AS bad_nonscore_identity,
  SUM(CASE WHEN ST_Load_All_w != (ST_Load_ScoreLinked_w + ST_Load_NonScore_w) THEN 1 ELSE 0 END) AS bad_partition_identity
FROM {PANEL_TABLE}
""").df()

print(st_identity)

bad_all = int(st_identity["bad_all_identity"].iloc[0])
bad_sl = int(st_identity["bad_scorelinked_identity"].iloc[0])
bad_ns = int(st_identity["bad_nonscore_identity"].iloc[0])
bad_part = int(st_identity["bad_partition_identity"].iloc[0])

if bad_all or bad_sl or bad_ns or bad_part:
    sample = con.execute(f"""
    SELECT
      {SEASON_COL} AS season,
      {WEEK_COL} AS week,
      {TEAM_COL} AS team,
      ST_Punt_w,
      ST_PuntReturn_w,
      ST_Kickoff_w,
      ST_KickReturn_w,
      ST_FG_w,
      ST_XP_w,
      ST_Rare_w,
      ST_Load_All_w,
      ST_Load_ScoreLinked_w,
      ST_Load_NonScore_w
    FROM {PANEL_TABLE}
    WHERE
      ST_Load_All_w != (ST_Punt_w + ST_PuntReturn_w + ST_Kickoff_w + ST_KickReturn_w + ST_FG_w + ST_XP_w + ST_Rare_w)
      OR ST_Load_ScoreLinked_w != (ST_Kickoff_w + ST_XP_w + ST_FG_w)
      OR ST_Load_NonScore_w != (ST_Punt_w + ST_PuntReturn_w + ST_KickReturn_w + ST_Rare_w)
      OR ST_Load_All_w != (ST_Load_ScoreLinked_w + ST_Load_NonScore_w)
    ORDER BY season DESC, week DESC, team
    LIMIT 50
    """).df()
    print(sample)
    raise RuntimeError("Special teams buckets do not satisfy the intended identities, investigate notebook 04 bucketing logic")

   bad_all_identity  bad_scorelinked_identity  bad_nonscore_identity  \
0               0.0                       0.0                    0.0   

   bad_partition_identity  
0                     0.0  


Quick sanity check to confirm that 'ScoreLinked' moves with the points environment more than 'NonScore' by comparing correlations with total game points

In [12]:
df_points = con.execute(f"""
SELECT
  {SEASON_COL} AS season,
  {WEEK_COL} AS week,
  {TEAM_COL} AS team,
  points_for,
  points_against,
  ST_Load_ScoreLinked_w,
  ST_Load_NonScore_w
FROM {PANEL_TABLE}
""").df()

df_points["total_points_game"] = df_points["points_for"] + df_points["points_against"]

df_corr = df_points.dropna(subset=["total_points_game", "ST_Load_ScoreLinked_w", "ST_Load_NonScore_w"]).copy()

corr_scorelinked = df_corr["ST_Load_ScoreLinked_w"].corr(df_corr["total_points_game"])
corr_nonscore = df_corr["ST_Load_NonScore_w"].corr(df_corr["total_points_game"])

print("corr ScoreLinked with total points game", corr_scorelinked)
print("corr NonScore with total points game", corr_nonscore)

if abs(corr_scorelinked) < abs(corr_nonscore):
    raise RuntimeError("ScoreLinked is not more correlated with points environment than NonScore, investigate ScoreLinked definition and joins")

by_season = (
    df_corr.groupby("season", as_index=False)
    .apply(
        lambda g: pd.Series({
            "corr_scorelinked_total_points": g["ST_Load_ScoreLinked_w"].corr(g["total_points_game"]),
            "corr_nonscore_total_points": g["ST_Load_NonScore_w"].corr(g["total_points_game"]),
            "n_rows": len(g),
        }),
        include_groups=False,
    )
    .reset_index(drop=True)
)

by_season = by_season.reset_index(drop=True)
by_season.sort_values("season").tail(15)

corr ScoreLinked with total points game 0.419912944416738
corr NonScore with total points game -0.04675335277819033


,season,corr_scorelinked_total_points,corr_nonscore_total_points,n_rows
0,2012,0.419281,0.026833,512.0
1,2013,0.406215,-0.030173,512.0
2,2014,0.383458,0.027097,512.0
3,2015,0.339905,0.085178,512.0
4,2016,0.318892,-0.170560,512.0
5,2017,0.456851,-0.029730,512.0
6,2018,0.471858,-0.043293,512.0
7,2019,0.413007,-0.064210,512.0
8,2020,0.445155,-0.076488,512.0
9,2021,0.435167,-0.074524,544.0


Quick sanity check to confirm that 'NonScore' correlates more with punts and returns than with touchdowns by constructing a touchdowns per team week table from play by play and comparing correlations

In [13]:
con.execute("DROP TABLE IF EXISTS team_week_tds_off")

con.execute("""
CREATE TABLE team_week_tds_off AS
SELECT
  season,
  week,
  CAST(posteam AS VARCHAR) AS team,
  SUM(CASE WHEN touchdown = 1 THEN 1 ELSE 0 END) AS tds_off_w
FROM pbp
WHERE posteam IS NOT NULL
GROUP BY 1,2,3
""")

con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT season || '-' || week || '-' || team) AS distinct_keys,
  MIN(tds_off_w) AS min_tds,
  MAX(tds_off_w) AS max_tds,
  AVG(tds_off_w) AS avg_tds
FROM team_week_tds_off
""").df()

df_ns = con.execute(f"""
SELECT
  p.{SEASON_COL} AS season,
  p.{WEEK_COL} AS week,
  p.{TEAM_COL} AS team,
  p.ST_Load_NonScore_w,
  p.ST_Punt_w,
  p.ST_PuntReturn_w,
  p.ST_KickReturn_w,
  COALESCE(t.tds_off_w, 0) AS tds_off_w
FROM {PANEL_TABLE} p
LEFT JOIN team_week_tds_off t
  ON t.season = p.{SEASON_COL}
 AND t.week = p.{WEEK_COL}
 AND t.team = CAST(p.{TEAM_COL} AS VARCHAR)
""").df()

df_ns["punt_return_proxy"] = df_ns["ST_Punt_w"] + df_ns["ST_PuntReturn_w"] + df_ns["ST_KickReturn_w"]

df_ns_corr = df_ns.dropna(subset=["ST_Load_NonScore_w", "punt_return_proxy", "tds_off_w"]).copy()

corr_ns_punts = df_ns_corr["ST_Load_NonScore_w"].corr(df_ns_corr["punt_return_proxy"])
corr_ns_tds = df_ns_corr["ST_Load_NonScore_w"].corr(df_ns_corr["tds_off_w"])

print("corr NonScore with punts and returns proxy", corr_ns_punts)
print("corr NonScore with offensive touchdowns", corr_ns_tds)

if abs(corr_ns_punts) < abs(corr_ns_tds):
    raise RuntimeError("NonScore is not more correlated with punts and returns than touchdowns, investigate NonScore definition and component mapping")

by_season_ns = (
    df_ns_corr.groupby("season", as_index=False)
    .apply(
        lambda g: pd.Series({
            "corr_nonscore_punts_returns": g["ST_Load_NonScore_w"].corr(g["punt_return_proxy"]),
            "corr_nonscore_tds_off": g["ST_Load_NonScore_w"].corr(g["tds_off_w"]),
            "n_rows": len(g),
        }),
        include_groups=False,
    )
    .reset_index(drop=True)
)

by_season_ns = by_season_ns.reset_index(drop=True)
by_season_ns.sort_values("season").tail(15)

corr NonScore with punts and returns proxy 0.9964768707917049
corr NonScore with offensive touchdowns 0.08403798455172516


,season,corr_nonscore_punts_returns,corr_nonscore_tds_off,n_rows
0,2012,0.998043,0.296740,512.0
1,2013,0.997816,0.309269,512.0
2,2014,0.997503,0.301262,512.0
3,2015,0.997496,0.377961,512.0
4,2016,0.996235,0.192567,512.0
5,2017,0.995994,0.061211,512.0
6,2018,0.995881,0.051474,512.0
7,2019,0.995465,0.031666,512.0
8,2020,0.993027,-0.182803,512.0
9,2021,0.994454,-0.264015,544.0


Quick sanity check to confirm that the next week modeling view has no unexpected nulls in core exposures and controls, which protects against join failures that can potentially collapse sample size later

In [14]:
core_cols = [
    "ST_Load_NonScore_w",
    "ST_Load_ScoreLinked_w",
    "ST_Load_All_w",
    "Z_ST_NonScore_w",
    "ST_Shock_NonScore_w",
    "ST_Vol_NonScore_w",
    "Cum_Shocks_NonScore_w",
    "offensive_snaps_w",
    "defensive_snaps_w",
    "off_yards_per_play_w",
    "points_for",
    "points_against",
    "score_diff_w",
    "blowout_flag_w",
    "days_rest_w",
    "short_week_flag_w",
    "bye_last_week_flag_w",
    "home_flag_w",
    "Inj_Off_Next_w",
    "Inj_Def_Next_w",
    "Inj_Off_Last_w",
    "Inj_Def_Last_w",
]

cols_now = set(_existing_cols("team_week_panel_nextweek_model"))
core_cols = [c for c in core_cols if c in cols_now]

null_expr = ",\n  ".join([f"SUM(CASE WHEN {c} IS NULL THEN 1 ELSE 0 END) AS null_{c}" for c in core_cols])

nulls = con.execute(f"""
SELECT
  COUNT(*) AS rows_model,
  {null_expr}
FROM team_week_panel_nextweek_model
""").df()

nulls

bad = []
for c in core_cols:
    v = int(nulls[f"null_{c}"].iloc[0])
    if v != 0:
        bad.append((c, v))

print("Nonzero null counts in model view", bad)

if bad:
    raise RuntimeError("Model view contains nulls in required exposures or controls, rerun upstream joins and inspect the listed columns")

Nonzero null counts in model view []


We aggregate the results of our structural and integrity tests into a single, high-level summary table, providing a centralized "health report" for the dataset before the final regression begins

In [15]:
con.execute("DROP TABLE IF EXISTS panel_step11_diagnostics")

con.execute(f"""
CREATE TABLE panel_step11_diagnostics AS
SELECT
  CURRENT_TIMESTAMP AS created_at,
  (SELECT COUNT(*) FROM {PANEL_TABLE}) AS rows_panel,
  (SELECT COUNT(*) FROM team_week_panel_nextweek_model) AS rows_nextweek_model,
  (SELECT COUNT(*) FROM {PANEL_TABLE}) - (SELECT COUNT(*) FROM team_week_panel_nextweek_model) AS rows_dropped_no_next_week,
  {float(corr_scorelinked)} AS corr_scorelinked_total_points,
  {float(corr_nonscore)} AS corr_nonscore_total_points,
  {float(corr_ns_punts)} AS corr_nonscore_punts_returns,
  {float(corr_ns_tds)} AS corr_nonscore_tds_off
""")

con.execute("SELECT * FROM panel_step11_diagnostics").df()

,created_at,rows_panel,rows_nextweek_model,rows_dropped_no_next_week,corr_scorelinked_total_points,corr_nonscore_total_points,corr_nonscore_punts_returns,corr_nonscore_tds_off
0,2026-01-05 19:44:29.192408-05:00,6782,5950,832,0.419913,-0.046753,0.996477,0.084038


Quick sanity check to confirm that the rows missing 'w+1' behave like expected at the team season level, and that the 2017 and 2022 deviations are not a broken panel join

In [16]:
missing_dist = con.execute("""
WITH per_team AS (
  SELECT
    season,
    team_key,
    SUM(CASE WHEN has_next_week = 0 THEN 1 ELSE 0 END) AS n_no_next_week,
    COUNT(*) AS n_rows
  FROM panel_next_week_flags
  GROUP BY 1,2
)
SELECT
  season,
  n_no_next_week,
  COUNT(*) AS n_teams
FROM per_team
GROUP BY 1,2
ORDER BY season, n_no_next_week
""").df()

print(missing_dist)

odd_teams = con.execute("""
WITH per_team AS (
  SELECT
    season,
    team_key,
    SUM(CASE WHEN has_next_week = 0 THEN 1 ELSE 0 END) AS n_no_next_week,
    COUNT(*) AS n_rows
  FROM panel_next_week_flags
  GROUP BY 1,2
)
SELECT *
FROM per_team
WHERE n_no_next_week NOT IN (1, 2, 3)
ORDER BY season, team_key
""").df()

print("team seasons with unexpected n_no_next_week", len(odd_teams))
if len(odd_teams) != 0:
    print(odd_teams.head(50))
    raise RuntimeError("Unexpected missing next week pattern, investigate schedule ingestion, canceled games, or missing team weeks")

    season  n_no_next_week  n_teams
0     2012             2.0       32
1     2013             2.0       32
2     2014             2.0       32
3     2015             2.0       32
4     2016             2.0       32
5     2017             1.0        2
6     2017             2.0       30
7     2018             2.0       32
8     2019             2.0       32
9     2020             2.0       32
10    2021             2.0       32
11    2022             2.0       30
12    2022             3.0        2
13    2023             2.0       32
14    2024             2.0       32
team seasons with unexpected n_no_next_week 0


Quick sanity check to confirm that the special teams bucket identities cannot pass if any contributing columns are null

In [17]:
st_identity_hardened = con.execute(f"""
SELECT
  SUM(
    CASE
      WHEN ST_Load_All_w IS NULL
        OR ST_Punt_w IS NULL
        OR ST_PuntReturn_w IS NULL
        OR ST_Kickoff_w IS NULL
        OR ST_KickReturn_w IS NULL
        OR ST_FG_w IS NULL
        OR ST_XP_w IS NULL
        OR ST_Rare_w IS NULL
      THEN 1
      WHEN ST_Load_All_w != (ST_Punt_w + ST_PuntReturn_w + ST_Kickoff_w + ST_KickReturn_w + ST_FG_w + ST_XP_w + ST_Rare_w)
      THEN 1
      ELSE 0
    END
  ) AS bad_all_identity,
  SUM(
    CASE
      WHEN ST_Load_ScoreLinked_w IS NULL
        OR ST_Kickoff_w IS NULL
        OR ST_XP_w IS NULL
        OR ST_FG_w IS NULL
      THEN 1
      WHEN ST_Load_ScoreLinked_w != (ST_Kickoff_w + ST_XP_w + ST_FG_w)
      THEN 1
      ELSE 0
    END
  ) AS bad_scorelinked_identity,
  SUM(
    CASE
      WHEN ST_Load_NonScore_w IS NULL
        OR ST_Punt_w IS NULL
        OR ST_PuntReturn_w IS NULL
        OR ST_KickReturn_w IS NULL
        OR ST_Rare_w IS NULL
      THEN 1
      WHEN ST_Load_NonScore_w != (ST_Punt_w + ST_PuntReturn_w + ST_KickReturn_w + ST_Rare_w)
      THEN 1
      ELSE 0
    END
  ) AS bad_nonscore_identity,
  SUM(
    CASE
      WHEN ST_Load_All_w IS NULL
        OR ST_Load_ScoreLinked_w IS NULL
        OR ST_Load_NonScore_w IS NULL
      THEN 1
      WHEN ST_Load_All_w != (ST_Load_ScoreLinked_w + ST_Load_NonScore_w)
      THEN 1
      ELSE 0
    END
  ) AS bad_partition_identity
FROM {PANEL_TABLE}
""").df()

print(st_identity_hardened)

bad_any = int(st_identity_hardened[["bad_all_identity","bad_scorelinked_identity","bad_nonscore_identity","bad_partition_identity"]].sum(axis=1).iloc[0])
if bad_any != 0:
    raise RuntimeError("Special teams bucket identities failed or encountered null inputs, investigate notebook 04 bucketing and joins")

   bad_all_identity  bad_scorelinked_identity  bad_nonscore_identity  \
0               0.0                       0.0                    0.0   

   bad_partition_identity  
0                     0.0  
